In [ ]:
import json
import geopandas as gpd
import pandas as pd
import math
from shapely.geometry import Point, MultiPolygon
from descartes import PolygonPatch

import csv
import numpy as np
import random
import matplotlib.pyplot as plt

import os


In [ ]:
demographics = pd.read_csv("../demographics.csv")
demographics['wardIndex'] = demographics['wardIndex'].astype(int)
demographics['wardNo'] = demographics['wardNo'].astype(int)
demographics = demographics.sort_values('wardIndex').reset_index(drop=True)

geoDF = gpd.read_file("../city.geojson")
geoDF['wardNo'] = geoDF['wardNo'].astype(int)
geoDF['wardIndex'] = geoDF['wardNo'] - 1
geoDF = geoDF[['wardIndex','wardNo', 'wardName', 'geometry']]
geoDF['wardBounds'] = geoDF.apply(lambda row: MultiPolygon(row['geometry']).bounds, axis=1)
geoDF.sort_values('wardIndex', inplace=True)
geoDF = geoDF.reset_index(drop=True)

In [ ]:
print(f"{'wardIndex'.ljust(10)}{'city.geojson'.ljust(20)}{'demographics.csv'.ljust(20)}")
for i in range(geoDF.shape[0]):
    if geoDF["wardName"].iloc[i] != demographics['wardName'].iloc[i]:
        print(f"\nWARNING: Check if this is a mismatch!")
    print(f"{str(i).ljust(10)}{geoDF['wardName'].iloc[i].ljust(20)}{demographics['wardName'].iloc[i].ljust(20)}") 
        


In [ ]:
geoDFslums = gpd.read_file('slumClusters.geojson')
wardslums = [[] for _ in range(len(geoDF))]
for i in range(len(geoDFslums)):
    for j in range(len(geoDF)):
        if geoDFslums["geometry"].iloc[i].intersects(geoDF["geometry"].iloc[j]):
            wardslums[j].append(i)

In [ ]:
for w in range(nwards):
    fig = plt.figure(figsize=(15,10))
    plt.title(f"{demographics['wardName'].iloc[w]}")
    ax = fig.gca() 
    ax.add_patch(PolygonPatch(geoDF["geometry"].iloc[w], fc="blue", ec="blue", alpha=0.1, zorder=2 ))
    for s in wardslums[w]:
        ax.add_patch(PolygonPatch(geoDFslums["geometry"].iloc[s], fc="red", ec="red", zorder=2 ))
    ax.axis('scaled')
    fig.show()

In [ ]:
def sampleRandomLatLong(wardIndex):
    #IMPORTANT: geoDF uses (lon, lat) order
    (lon1,lat1,lon2,lat2) = geoDF['wardBounds'].iloc[wardIndex]
    while True:
        lat = random.uniform(lat1,lat2)
        lon = random.uniform(lon1,lon2)
        point = Point(lon,lat)
        if MultiPolygon(geoDF['geometry'].iloc[wardIndex]).contains(point):
            return (lat,lon)
        
def sampleRandomLatLong_hd(wardIndex):
    while True:
        (lat,lon) = sampleRandomLatLong(wardIndex)
        point = Point(lon,lat) #IMPORTANT: Point takes in order of longitude, latitude
        
        if len(wardslums[wardIndex])==0:
            #No designated hd areas in this ward. Return random point
            return (lat,lon)
        else:
            for i in wardslums[wardIndex]:
                if geoDFslums["geometry"].iloc[i].contains(point):
                    return (lat,lon)

In [ ]:
numpoints = 1000
nwards = 24
for i in range(nwards):
    print(f"Generating points for ward {i+1}")

    fname=str(i)+".csv"
    print(f"{fname}:")
    with open(fname, mode='w+') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for count in range(numpoints):
            (lat,lon) = sampleRandomLatLong(i)
            writer.writerow([str(lat),str(lon)])
            if count % 20 == 0:
                print(".",end='')
    print("")

In [ ]:
for i in range(nwards):
    print(f"Generating points for ward {i+1} hd_area")

    fname=str(i + nwards)+".csv"
    print(f"{fname}:")
    with open(fname, mode='w+') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for count in range(numpoints):
            (lat,lon) = sampleRandomLatLong_hd(i)
            writer.writerow([str(lat),str(lon)])
            if count % 20 == 0:
                print(".",end='')
    print("")


In [ ]:
for w in range(nwards):
    df_points = pd.read_csv(f"{w}.csv", header=None, names=["lat", "lon"])
    df_points_hd = pd.read_csv(f"{w + nwards}.csv", header=None, names=["lat", "lon"])

    fig = plt.figure(figsize=(15,10))
    plt.title(f"{demographics['wardName'].iloc[w]}")
    ax = fig.gca() 
    ax.add_patch(PolygonPatch(geoDF["geometry"].iloc[w], fc="blue", ec="blue", alpha=0.1, zorder=1 ))
    for s in wardslums[w]:
        ax.add_patch(PolygonPatch(geoDFslums["geometry"].iloc[s], fc="red", ec="red", zorder=2 ))
    ax.scatter(df_points['lon'], df_points['lat'], c="blue", alpha=0.5,zorder=3)
    ax.scatter(df_points_hd['lon'], df_points_hd['lat'], c="yellow", alpha=0.5,zorder=4)

    ax.axis('scaled')
    fig.show()